# JuliaのTips集
このページはJuliaでの実装におけるTips (詰まったところの解決策)をまとめたものである。体系的にまとまってはいない。

## 1. 関数名の!記号
単なる**慣習**として関数への入力を変更する場合に!を付ける。

関数内で配列を変更する場合には注意が必要である。以下に入力された配列を同じサイズの要素1の配列で置き換える、ということを目的として書かれた2つの関数がある。違いは`v`の後に`[:]`としているかどうかである。

In [24]:
function wrong!(A::Array)
    a = ones(size(a))
end

function right!(a::Array)
    a[:] = ones(size(a))
end

right! (generic function with 2 methods)

実行すると`wrong!`の場合には入力された配列が変更されていないことがわかる。

In [28]:
using Random
v = rand(2, 2)
print("v : ", v)

wrong!(v)
print("\nwrong : ", v)

right!(v)
print("\nright : ", v)

v : [0.7076135447844478 0.055541893053967284; 0.1206100612228862 0.18385182053893634]
wrong : [0.7076135447844478 0.055541893053967284; 0.1206100612228862 0.18385182053893634]
right : [1.0 1.0; 1.0 1.0]

## 2. 配列の1次元化
配列を一次元化(flatten)する方法。まずは3次元配列を作成する。

In [7]:
B = rand(2, 2, 2)

2×2×2 Array{Float64,3}:
[:, :, 1] =
 0.997107  0.0827965
 0.394606  0.847174

[:, :, 2] =
 0.0305417  0.448929
 0.270553   0.601419

用意されている`flatten`を素直に用いると次のようになる。

In [2]:
import Base.Iterators: flatten
collect(flatten(B))

8-element Array{Float64,1}:
 0.25323335359382226
 0.8475284309218816
 0.46170218524594486
 0.3908851101558619
 0.2049406129688014
 0.20068478557126213
 0.29072826846396627
 0.5193322837086369

単に`B[:]`とするだけでもよい。

In [3]:
B[:]

8-element Array{Float64,1}:
 0.25323335359382226
 0.8475284309218816
 0.46170218524594486
 0.3908851101558619
 0.2049406129688014
 0.20068478557126213
 0.29072826846396627
 0.5193322837086369

## 3. 行列の行・列ごとの正規化
シミュレーションにおいてニューロン間の重み行列を行あるいは列ごとに正規化 (weight normalization)する場合がある。これは各ニューロンへの入力の大きさを同じにする働きや重みの発散を防ぐ役割がある。以下では行ごとの和を1にする。

In [8]:
W = rand(3,3)

3×3 Array{Float64,2}:
 0.166781  0.0552745  0.973622
 0.100336  0.753766   0.370005
 0.378222  0.165159   0.471515

In [9]:
Wnormed = W ./ sum(W, dims=1)

3×3 Array{Float64,2}:
 0.258439  0.0567384  0.536389
 0.155479  0.773729   0.203843
 0.586082  0.169533   0.259767

In [10]:
print(sum(Wnormed, dims=1))

[1.0 1.0 1.0]

## 4. 行列の結合 (concatenate)
行列の結合はMATLABに近い形式で行うことができる。まず、2つの行列A, Bを用意する。

In [1]:
A = [1 2; 3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [2]:
B = [4 5 6; 7 8 9]

2×3 Array{Int64,2}:
 4  5  6
 7  8  9

### 4.1 水平結合 (Horizontal concatenation)
`hcat`を使うやり方と、`[ ]`を使うやり方がある。

In [13]:
H1 = hcat(A,B)

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

In [11]:
H2 = [A B]

2×5 Array{Int64,2}:
 1  2  4  5  6
 3  4  7  8  9

なお、MATLABのように次のようにすると正しく結合はされない。

In [16]:
H3 = [A, B]

2-element Array{Array{Int64,2},1}:
 [1 2; 3 4]
 [4 5 6; 7 8 9]

### 4.2 垂直結合 (Vertical concatenation)

In [6]:
V1 = vcat(A, B')

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [19]:
V2 = [A; B']

5×2 Array{Int64,2}:
 1  2
 3  4
 4  7
 5  8
 6  9

In [21]:
[V2 [A;B']]

5×4 Array{Int64,2}:
 1  2  1  2
 3  4  3  4
 4  7  4  7
 5  8  5  8
 6  9  6  9